In [3]:
import re, subprocess, boto3, json, shlex, mysql, os, urllib
import pandas as pd
import numpy as np
from s3path import S3Path
from pathlib import Path
from tqdm.notebook import tqdm
from packaging import version
pd.set_option("display.max_colwidth", 40)

# Define Helper Functions

In [4]:
# Map from species to S3 locations of annotation and fasta files
ref_map = {
    ("GRCh38", 85):
    (
        "s3://seqc-public/genomes/hg38_long_poly/annotations.gtf",
        "s3://seqc-public/genomes/hg38_long_poly/homo_sapiens.fa",
    ),
    ("GRCm38", 87):
    (
        "s3://seqc-public/genomes/mm38_long_polya/annotations.gtf",
        "s3://seqc-public/genomes/mm38_long_polya/mus_musculus.fa",
    ),
    ("GRCh38", 100):
    (
        "s3://seqc-public/genomes/GRCh38-Ensembl-100/annotations.gtf",
        "s3://seqc-public/genomes/GRCh38-Ensembl-100/homo_sapiens.fa",
    ),
    ("GRCm38", 100):
    (
        "s3://seqc-public/genomes/GRCm38-Ensembl-100/annotations.gtf",
        "s3://seqc-public/genomes/GRCm38-Ensembl-100/mus_musculus.fa",
    ),
    #(('Human', 'Mouse'), ('GRCh38', 'GRCm38'), 100), : No FASTA as of 2021-01-15
}

# Map from species column of form to Ensembl prefix
ensembl_map = {
    "Human": "ENS",
    "Mouse": "ENSMUS",
    ('Human', 'Mouse'): "ENS",
}

In [5]:
def parse_species(response):
    
    species_sub, ensembl_sub = response.split("/")

    # Get species and species id
    species_pattern = r'([a-zA-Z]*) \((.*?)\)'
    species_matches = re.findall(species_pattern, species_sub)
    if len(species_matches)>1:
        names = (match[0] for match in species_matches)
        ids = (match[1] for match in species_matches)
        species = (names, ids)
    else: 
        species = species_matches[0]

    # Get ensembl version
    ensembl_pattern = r'Ensembl (\d*)'
    ensembl = re.findall(ensembl_pattern, ensembl_sub)[0]

    return species + (int(ensembl),)

In [6]:
# FASTQ reads/indices required for each workflow
fastq_map = {
    'Hashtag': ['R1', 'R2'],
    'CiteSeq': ['R1', 'R2'],
    'AsapSeq': ['R1', 'R2', 'R3'],
    'CellRangerATAC': ['I1', 'R1', 'R2', 'R3'],
}

In [7]:
# Get fastq file paths on S3 for each file id
# Returns dictionary from id to s3 path
# Throws exception if FASTQs don't exist for any id
def get_fastqs(
    path: str, # path to directory containing FASTQ files
    fastq_file_ids: list, # FASTQ file ids needed for this run type (e.g. I1, R1, R2, etc.)
):
    fastq_map = dict()
    _, bucket, key, _, _ = urllib.parse.urlsplit(path)
    for fid in fastq_file_ids:
        files = get_s3_objects(
            bucket, key.lstrip("/"),
            re.compile(f"_{fid}_\d{{3}}.fastq.gz$")
        )
        try:
            assert files, f"AssertionError: Missing `{fid}` archives!"
            fastq_map[fid] = [os.path.join("s3://", bucket, str(f)) for f in files]
        except AssertionError as err:
            logging.warning("%s\n\t %s", err, path)
            return
    return fastq_map

In [8]:
# Extract FASTQ sample name from list of files
# Note: FASTQ name is file name up to lane id (e.g. L001, L002, etc.)
def get_fastqs_name(fastqs):
    fastq_name_re = r".*/(.*)_S\d+_L\d{3}_[A-Za-z]\d_\d{3}.fastq.gz$"
    fastq_names = [re.match(fastq_name_re, x)[1] for x in fastqs]
    assert len(set(fastq_names)) == 1 # make sure all names are same
    return fastq_names[0]

In [9]:
# Numpy encoder for JSON from pandas series
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [10]:
# from SCRIdb
def get_s3_objects(bucket, key, pattern, full_uri=False):
    
    s3r = boto3.resource("s3")
    bucket_s3 = s3r.Bucket(bucket)
    objects = []
    for obj in bucket_s3.objects.filter(Prefix=key):
        hit = pattern.search(obj.key)
        if hit:
            objects.append(obj.key)
    if full_uri:
        objects = [f"s3://{bucket}/{o}" for o in objects]
    return objects

In [11]:
def execute_query(query, user, password):
    with connect(
        host="peer-lab-db.cggxmlwgzzpw.us-east-1.rds.amazonaws.com",
        database="peer_lab_db",
        user=user,
        password=password,
    ) as connection:
        with connection.cursor(buffered=True) as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
    return result

In [12]:
# Get species from database for given sample
from mysql.connector import connect, Error

def get_species(sample_id, user, password):
    try:
        table_sample_data = "peer_lab_db.sample_data"
        table_species = "peer_lab_db.species"
        table_genome_idx = "peer_lab_db.genome_index"
        query = f"""
        SELECT {table_species}.Species
        FROM {table_species}
        LEFT JOIN {table_genome_idx}
        ON {table_species}.id = {table_genome_idx}.species_id
        LEFT JOIN {table_sample_data}
        ON {table_genome_idx}.id = {table_sample_data}.genomeIndex_id
        WHERE {table_sample_data}.id = {sample_id}
        """
        result = execute_query(query, user, password)[0][0]
        return result
    except Error as e:
        print(f"Error: {e}")

In [13]:
# Get species from database for given sample
from mysql.connector import connect, Error

def get_sc_tech(sample_id, user, password):
    try:
        table_sample_data = "peer_lab_db.sample_data"
        table_sc_tech = "peer_lab_db.sc_tech"
        table_genome_idx = "peer_lab_db.genome_index"
        query = f"""
        SELECT {table_sc_tech}.sc_Tech
        FROM {table_sc_tech}
        LEFT JOIN {table_genome_idx}
        ON {table_sc_tech}.id = {table_genome_idx}.scTech_id
        LEFT JOIN {table_sample_data}
        ON {table_genome_idx}.id = {table_sample_data}.genomeIndex_id
        WHERE {table_sample_data}.id = {sample_id}
        """
        result = execute_query(query, user, password)[0][0]
        return result
    except Error as e:
        print(f"Error: {e}")

In [14]:
# Get species from database for given sample
from mysql.connector import connect, Error

def get_sample_id(sample_name, user, password):
    try:
        table_sample_data = "peer_lab_db.sample_data"
        query = f"""
        SELECT {table_sample_data}.id
        FROM {table_sample_data}
        WHERE {table_sample_data}.Sample="{sample_name}"
        """
        result = execute_query(query, user, password)[0][0]
        return result
    except Error as e:
        print(f"Error: {e}")

In [15]:
# Get species from database for given sample
from mysql.connector import connect, Error

def get_project_id(sample_id, user, password):
    try:
        table_sample_data = "peer_lab_db.sample_data"
        table_project_data = "peer_lab_db.project_data"
        query = f"""
        SELECT {table_project_data}.projectName
        FROM {table_project_data}
        LEFT JOIN {table_sample_data}
        ON {table_project_data}.id = {table_sample_data}.projectData_id
        WHERE {table_sample_data}.id = {sample_id}
        """
        result = execute_query(query, user, password)[0][0]
        return result
    except Error as e:
        print(f"Error: {e}")

In [16]:
def get_SEQC_version(loc):
    try:
        cmd = f"aws s3 cp {loc}/seqc-results/seqc_log.txt -"
        out = subprocess.run(shlex.split(cmd), universal_newlines=True, capture_output=True).__dict__["stdout"]
        version = re.match(r".*SEQC=v(\d+\.\d+\.\d+).*", out)[1]
        return version
    except:
        return "N/A"

In [17]:
def get_file_prefix(loc):
    try:
        cmd = f"aws s3 ls {loc}/seqc-results/"
        out = subprocess.run(shlex.split(cmd), universal_newlines=True, capture_output=True).__dict__["stdout"]
        
        # Note: I'm expecting the aligned bam file to be in loc
        bam_pattern = re.compile(r"(.*)_Aligned\.out\.bam$")
        filename = list(filter(bam_pattern.match, out.split()))[0]
        file_prefix = re.match(bam_pattern, filename)[1]
        return file_prefix
    except:
        raise ValueError(f"BAM file not found in {loc}")
        return ""

In [18]:
# FASTQ reads/indices required for each workflow
# Shoudl replace with JSON file
cr_reference_map = {
    'CellRangerArc':
    {
        'Human': "https://cf.10xgenomics.com/supp/cell-arc/refdata-cellranger-arc-GRCh38-2020-A.tar.gz",
        'Mouse': "https://cf.10xgenomics.com/supp/cell-arc/refdata-cellranger-arc-mm10-2020-A-2.0.0.tar.gz",
    },
    'CellRangerATAC':
    {
        'Human': "https://cf.10xgenomics.com/supp/cell-atac/refdata-cellranger-arc-GRCh38-2020-A-2.0.0.tar.gz",
        'Mouse': "https://cf.10xgenomics.com/supp/cell-atac/refdata-cellranger-arc-mm10-2020-A-2.0.0.tar.gz",
    }
}

def get_cr_reference(sample_id, prefix, user, password):
    # Get species from database to decide reference
    species = get_species(sample_id, user, password)
    
    # Map to reference locations
    try:
        return cr_reference_map[prefix][species]
    except:
        raise ValueError(f"Unknown Species: {species}")

In [19]:
def get_bc_whitelist(sample_id):
    # Get version from database to decide whitelist
    sc_tech = get_sc_tech(sample_id, creds["user"], creds["password"])
    
    # Map to reference locations
    if "V3" in sc_tech:
        return "s3://seqc-public/barcodes/ten_x_v3/flat/3M-february-2018.txt"
    elif "V2" in sc_tech:
        return "s3://seqc-public/barcodes/ten_x_v2/flat/737K-august-2016.txt"
    else:
        raise ValueError(f"Unknown Technology: {sc_tech}")

In [20]:
def run(
    workflow_path: str,
    execp: str,
    secrets: str,
    inputs: str,
    labels: str,
    options: str,
):
    # change working directory to the pipeline package
    oldwd = os.getcwd()
    os.chdir(workflow_path)
    
    # execute the pipeline command
    cmd = f"{workflow_path}/{execp} -k {secrets} -i {inputs} -l {labels} -o {options}"
    var = subprocess.run(shlex.split(cmd), universal_newlines=True, capture_output=True)
    out = var.__dict__
    
    # change working directory back
    os.chdir(oldwd)
    
    return out

# Process Samples

## Setup

In [21]:
# Maps from .wdl name (prefix) to results dirname
results_dirs = {
    "TransgenesSeqc": "refdata-seqc",
    "TransgenesCellRanger": "refdata-cellranger",
}

# Maps from .wdl name (prefix) to shell script
sh_files = {
    "TransgenesSeqc": "submit-seqc.sh",
    "TransgenesCellRanger": "submit-cellranger.sh",
}

# Version (CellRanger or Star) to use; Should be in database
latest_STAR = "2.5.3a"
latest_CellRanger = "6.1.1"

In [22]:
# Location of docker files
common_docker_registry = "quay.io/hisplan"

prefix = "TransgenesCellRanger" # Workflow to run; also .wdl filename prefix
pipeline_type = prefix # field in *.labels.json
output_dirname = results_dirs[prefix]

# If need to add comment, put here
comment = ""

In [23]:
# Locations of workflow-related directories and files
path_to_cromwell_secrets = f"{Path.home()}/.cromwell/cromwell-secrets.json" # CHANGE THIS
workflow_dir = f"{Path.home()}/scing/bin/transgenes-0.0.8" # CHANGE THIS
exec_file = sh_files[prefix]
config_dir = f"{workflow_dir}/config"
path_to_options = f"{workflow_dir}/Transgenes.options.aws.json"

# Other file locations
db_credentials_path = f"{Path.home()}/.config.json" # CHANGE THIS

In [24]:
# Set credentials based on SCRIdb CLI config file
with open(db_credentials_path) as f:
    creds = json.load(f)

In [25]:
# Location of CSV file containing Google Forms results
# Note: Script will build a transgene for EVERY row in CSV
path_to_forms = f"{Path.home()}/scing/transgene_forms/example.csv"

## Execution

In [26]:
# Get answers from forms submissions
submissions = pd.read_csv(
    path_to_forms, index_col = 3, dtype=str
)
submissions.index = submissions.index.rename("Sample")
submissions["Sample_ID"] = pd.Series(submissions.index).apply(
    lambda x: get_sample_id(x, creds['user'], creds['password'])
).values
submissions["Project_Name"] = submissions["Sample_ID"].apply(
    lambda x: get_project_id(x, creds['user'], creds['password'])
).values
submissions["Parsed_Species"] = submissions["Species"].apply(parse_species)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/moormana/scing/transgene_forms/example.csv'

In [ ]:
submissions

In [ ]:
# Load minimum inputs and labels fields from templates
pipeline_type = prefix.split('Transgenes')[-1].lower()
with open(f"{config_dir}/template.{pipeline_type}.inputs.json") as f:
    std_inputs_fields = list(json.load(f).keys())
    
with open(f"{config_dir}/template.{pipeline_type}.labels.json") as f:
    std_labels_fields = list(json.load(f).keys())
    
# Annotate all samples with workflow inputs and labels
inputs = pd.DataFrame(index=submissions.index, columns=std_inputs_fields,)
labels = pd.DataFrame(index=submissions.index, columns=std_labels_fields,)

# Annotate inputs
inputs[f"{prefix}.referenceName"] = \
    submissions["Project_Name"] + "-" + \
    submissions["Parsed_Species"].apply(lambda x: f"{x[1]}-Ensembl-{x[2]}") + "-" \
    "transgenes"
inputs[f"{prefix}.genomeReferenceFasta"] = \
    submissions["Parsed_Species"].apply(lambda x: ref_map[x[1:]][1])
inputs[f"{prefix}.annotationGtf"] = \
    submissions["Parsed_Species"].apply(lambda x: ref_map[x[1:]][0])
inputs[f"{prefix}.customFastaFiles"] = \
     submissions["AWS S3 location(s) of the FASTA sequence of your reporter genes"].apply(lambda x: x.split("\n"))
inputs[f"{prefix}.ensembleIdPrefix"] = \
    submissions["Parsed_Species"].apply(lambda x: ensembl_map[x[0]])
inputs[f"{prefix}.ensembleIds"] = submissions["Ensembl ID of transgene"].apply(lambda x: x.split("\n"))
if pipeline_type == "TransgenesSEQC":
    inputs[f"{prefix}.starVersion"] = latest_STAR
else:
    inputs[f"{prefix}.cellRangerVersion"] = latest_CellRanger
standard_biotypes = [
    "protein_coding", "lincRNA", "antisense",
    "IG_V_gene", "IG_D_gene", "IG_J_gene", "IG_C_gene",
    "TR_V_gene", "TR_D_gene", "TR_J_gene", "TR_C_gene",
]
inputs[f"{prefix}.biotypes"] = inputs.iloc[:, 0].apply(lambda x: standard_biotypes)
inputs[f"{prefix}.dockerRegistry"] = common_docker_registry

# Annotate labels
labels["pipelineType"] = prefix
labels["project"] = submissions["Project_Name"]
labels["sample"] = labels.index
labels["owner"] = creds["user"]
labels["destination"] = \
    submissions["AWS S3 location where you want the newly built genome to be stored"].str.strip("/") + \
    "/" + output_dirname
    
labels["transfer"] = "-"
labels["comment"] = creds["user"]

assert (std_inputs_fields == list(inputs.columns)) & (inputs.notna().values.all())
assert (std_labels_fields == list(labels.columns)) & (labels.notna().values.all())

In [2]:
inputs

NameError: name 'inputs' is not defined

In [40]:
labels

,pipelineType,project,sample,owner,destination,transfer,comment
Sample,,,,,,,
LJ-1422_PS19,TransgenesCellRanger,3PS19_SNSEQ,LJ-1422_PS19,moormana,s3://dp-lab-data/collaborators/yuemi...,-,moormana


In [41]:
stdouts = [] # to store all outputs
process = True

with tqdm(submissions.index) as t:

    for sample_name in t:

        # Write inputs and labels to file
        path_to_inputs = f"{config_dir}/{sample_name}.inputs.json"
        with open(path_to_inputs, "w") as f_inputs:
            json.dump(inputs.loc[sample_name].to_dict(), f_inputs, indent=4, cls=NpEncoder)

        path_to_labels = f"{config_dir}/{sample_name}.labels.json"
        with open(path_to_labels, "w") as f_labels:
            json.dump(labels.loc[sample_name].to_dict(), f_labels, indent=4, cls=NpEncoder)

        if process:
            stdouts.append(run(
                workflow_path = workflow_dir,
                execp = exec_file,
                secrets = path_to_cromwell_secrets,
                inputs = path_to_inputs,
                labels = path_to_labels,
                options = path_to_options,
            ))

  0%|          | 0/1 [00:00<?, ?it/s]

In [73]:
configs = !ls configs/
pattern = re.compile(r'3.*CPL.*')
configs = np.sort(list(filter(pattern.match, configs)))
l1 = configs[:-1:2]
l2 = configs[1::2]
for inputs, labels in zip(l1, l2):
    !bash submit-cellplex.sh -i configs/$inputs -l configs/$labels -k ../../../.cromwell/cromwell-secrets.json -o Sharp.options.aws.json

{"id":"6de3bcc1-8707-4047-890e-db16005688f9","status":"Submitted"}
{"id":"3a54f991-37b2-4c56-b447-e68d854d3246","status":"Submitted"}
{"id":"1964ed8b-33d6-4f5e-81b8-2eae34c1f437","status":"Submitted"}
{"id":"29c83f7e-4ce2-41bb-a0d6-d6143b2b53db","status":"Submitted"}
{"id":"3fed61b5-7d00-48e8-bb59-8f243a6a6702","status":"Submitted"}
{"id":"8e1b696e-53b8-4563-8a01-9f92d0941a33","status":"Submitted"}
